## Using API classes to get data and save to db

# <span style="color:red">clear all output before saving: db output contains passwords! </span>
 
- importing modules needed
- creates a temporary db using .env
- creates the API objects for each vendor/station type
- pulls data from the vendor API


In [1]:
%load_ext autoreload
%autoreload 2

from ewxpwsdb.db.database import Session, init_db, get_db_url, get_engine
from ewxpwsdb.db.models import WeatherStation, Reading, StationType
from ewxpwsdb.db.importdata import import_station_file, read_station_table
from sqlmodel import select

db_url = get_db_url()
print(db_url)
engine = get_engine(db_url)


postgresql+psycopg2://localhost:5432/ewxpws


### optional: create new temp database to work with

In [ ]:
init_db(engine)
s = Session(engine)
objects = [
    StationType(station_type= 'DAVIS' ),
    StationType(station_type= 'SPECTRUM' ), 
    StationType(station_type= 'ONSET'),
    StationType(station_type= 'RAINWISE'),
    StationType(station_type= 'ZENTRA'),
]
s.bulk_save_objects(objects)
s.commit()
station_file = '../data/test_stations.tsv'
# import, using the global 'engine' defined in database module
import_station_file(station_file)


**check that there are stations**

In [2]:
with Session(engine) as session:
        statement = select(WeatherStation)
        results = session.exec(statement)
        stations = results.all()
len(stations)

2024-01-30 22:55:07,797 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2024-01-30 22:55:07,798 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-01-30 22:55:07,799 INFO sqlalchemy.engine.Engine select current_schema()
2024-01-30 22:55:07,799 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-01-30 22:55:07,799 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2024-01-30 22:55:07,800 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-01-30 22:55:07,800 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-30 22:55:07,803 INFO sqlalchemy.engine.Engine SELECT weatherstation.id, weatherstation.station_code, weatherstation.station_type, weatherstation.install_date, weatherstation.timezone, weatherstation.ewx_user_id, weatherstation.lat, weatherstation.lon, weatherstation.location_description, weatherstation.api_config 
FROM weatherstation
2024-01-30 22:55:07,804 INFO sqlalchemy.engine.Engine [generated in 0.00032s] {}
2024-01-30 22:55:07,805 INFO sqlalchemy.engine.Engine RO

6

## Test APIs

 - Davis

In [4]:

from ewxpwsdb.weather_apis.weather_api import WeatherAPI, WeatherAPIConfig
from ewxpwsdb.weather_apis.davis_api import DavisAPI, DavisAPIConfig

In [5]:
# get a davis station
session = Session(engine)
statement = select(WeatherStation).where(WeatherStation.station_type == "DAVIS")
results = session.exec(statement)
davis_station = results.first()
station = DavisAPI(davis_station)
print(station)
session.close()

2024-01-30 22:55:25,468 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-01-30 22:55:25,470 INFO sqlalchemy.engine.Engine SELECT weatherstation.id, weatherstation.station_code, weatherstation.station_type, weatherstation.install_date, weatherstation.timezone, weatherstation.ewx_user_id, weatherstation.lat, weatherstation.lon, weatherstation.location_description, weatherstation.api_config 
FROM weatherstation 
WHERE weatherstation.station_type = %(station_type_1)s
2024-01-30 22:55:25,470 INFO sqlalchemy.engine.Engine [generated in 0.00042s] {'station_type_1': 'DAVIS'}
2024-01-30 22:55:25,471 INFO sqlalchemy.engine.Engine ROLLBACK


In [6]:
print(station.station_type)
print(station.sampling_interval)
print(station.APIConfigClass)

DAVIS
15
<class 'ewxpwsdb.weather_apis.davis_api.DavisAPIConfig'>


In [7]:
api_config = station.APIConfigClass.model_validate_json_str(davis_station.api_config)
print(api_config == station.api_config)

True


In [14]:
r = station.get_readings()


In [18]:
r[0].response_text

'{"code":"401","message":"API call timestamp parameter \\"t\\" is stale. The timestamp you provide must be within a range that is plus or minus 300 seconds of the API server\'s internal clock."}'

In [12]:

readings = station.transform(r)
readings

[]

## Clean up

remove test databases

In [ ]:
# if sqlite
import re
if re.match('sqlite', get_db_url()):
    from os import remove
    remove('ewxpws.db')

In [ ]:
# if postgresl
import re
from sqlmodel import delete, text
drop_stmt = text("""drop database ewxpws""")

if re.match('postgres', get_db_url()):
    with Session(engine) as session:
        session.exec(delete(Reading))
        session.exec(delete(StationType))
        session.exec(delete(WeatherStation))
        session.commit()

    # con = engine.connect()
    # con.execute(drop_stmt)
    # con.close()


In [ ]:
session.close()
engine.dispose()